# The [Modified Benedict-Webb-Rubin equation of state](https://en.wikipedia.org/wiki/Benedict–Webb–Rubin_equation) 
[Johnson, Zllweg, Gubbins, 
"The Lennard-Jones equation of state revisited",
Molecular Physics, 1993](Johnson_Lennard-Jones_1993.pdf)

implementing the parameters from Table 10

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [142]:
import itertools
from bokeh.palettes import Viridis6

In [139]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [66]:
import glob
import os.path as op

**Reduced Units:**
  - residual Helmholtz free energy: $A^*_r = A_r/N\epsilon$
  - pressure: $P^* = P\sigma^3/\epsilon$
  - temperature: 
  
| Quantity    | Reduced Units      | ------- Real Units --------      |
|-------------|--------------------|----------------------------------|
| temperature | $T^*=1$            | $T=119.8$ K                      |
| density     | $\rho^*=1.0$       | $\rho=1680$ kg/m$^3$             |
| time        | $\Delta t^*=0.005$ | $\Delta t=1.09\times 10^{-14}$ s |
| pressure    | $P^*=1$            | $P=41.9$ MPa                     |

$T^*=2.0$, to match Frenkel & Smit, figure 6.2

In [3]:
T = 2.0

Modified Benedict-Webb-Rubin (MBWR) equation of state, Johnson et al. 1993, eq. (7):
$$ P^* = \rho^* T^* + \sum_{i=1}^8 a_i \rho^{*(i+1)} + \exp\left(-\gamma\rho^{*2}\right)\sum_{i=1}^{6}b_i \rho^{*(2i+1)} $$

1 nonlinear parameter:
  - $\gamma = 3$

In [4]:
gamma = 3

In [5]:
parameters = np.loadtxt('mbwr_eos.dat')
x = np.empty(len(parameters)+1) 
x[1:] = parameters[:, 1] # use 1 indexing for clarity
# print x

First 19 parameters, $a_1, \dots, a_8$:
  - $a_1 = x_1 T^* + x_2 \sqrt{T} + x_3 + x_4/T^* + x_5/T^{*2}$
  - $a_2 = x_6 T^* + x_7 + x_8/T^* + x_9/T^{*2}$
  - $a_3 = x_{10} T^* + x_{11} + x_{12}/T^*$
  - $a_4 = x_{13}$
  - $a_5 = x_{14}/T^* + x_{15}/T^{*2}$
  - $a_6 = x_{16}/T^*$
  - $a_7 = x_{17}/T^* + x_{18}/T^{*2}$
  - $a_8 = x_{19}/T^{*2}$

In [6]:
a1 = x[1]*T + x[2]*np.sqrt(T) + x[3] + x[4]/T + x[5]/T**2
a2 = x[6]*T + x[7] + x[8]/T + x[9]/T**2
a3 = x[10]*T + x[11] + x[12]/T
a4 = x[13]
a5 = x[14]/T + x[15]/T**2
a6 = x[16]/T
a7 = x[17]/T + x[18]/T**2
a8 = x[19]/T**2
a = np.empty(8)
a[:] = [a1, a2, a3, a4, a5, a6, a7, a8]

Last 13 parameters, $b_1, \dots, b_6$:
  - $b_1 = x_{20}/T^{*2} + x_{21}/T^{*3}$
  - $b_2 = x_{22}/T^{*2} + x_{23}/T^{*4}$
  - $b_3 = x_{24}/T^{*2} + x_{25}/T^{*3}$
  - $b_4 = x_{26}/T^{*2} + x_{27}/T^{*4}$
  - $b_5 = x_{28}/T^{*2} + x_{29}/T^{*3}$
  - $b_6 = x_{30}/T^{*2} + x_{31}/T^{*3} + x_{32}/T^{*4}$

In [7]:
b1 = x[20]/T**2 + x[21]/T**3
b2 = x[22]/T**2 + x[23]/T**4
b3 = x[24]/T**2 + x[25]/T**3
b4 = x[26]/T**2 + x[27]/T**4
b5 = x[28]/T**2 + x[29]/T**3
b6 = x[30]/T**2 + x[31]/T**3 + x[32]/T**4
b = np.empty(6)
b[:] = [b1, b2, b3, b4, b5, b6]

Modified Benedict-Webb-Rubin (MBWR) equation of state, Johnson et al. 1993, eq. (7):
$$ P^* = \rho^* T^* + \sum_{i=1}^8 a_i \rho^{*(i+1)} + \exp\left(-\gamma\rho^{*2}\right)\sum_{i=1}^{6}b_i \rho^{*(2i+1)} $$

In [8]:
density = np.linspace(0.1, 0.9)
f = np.exp(-gamma * density**2)

In [9]:
pressure200 = (
    density * T + 
    a1 * density**2 + 
    a2 * density**3 +
    a3 * density**4 + 
    a4 * density**5 +
    a5 * density**6 +
    a6 * density**7 +
    a7 * density**8 +
    a8 * density**9 +
    f * b1 * density**3 +
    f * b2 * density**5 +
    f * b3 * density**7 +
    f * b4 * density**9 +
    f * b5 * density**11 +
    f * b6 * density**13    
)

Validating implentation against Frenkel Smit, Figure 6.2

In [11]:
eos = np.loadtxt('fs_eos_t2.csv', delimiter=',')
mc = np.loadtxt('fs_mc_t2.csv', delimiter=',')
md = np.loadtxt('fs_md_t2.csv', delimiter=',')

In [12]:
p = figure(width=500, height=500, x_axis_label='Density',
           y_axis_label='Pressure')
p.line(density, pressure200, legend='eos Johnson', line_width=2)
p.line(eos[:,0], eos[:,1], legend='eos F&S', line_dash=[2, 2], 
       line_width=2, color='firebrick')
p.circle(mc[:,0], mc[:,1], legend='MC F&S', size=12, 
         line_width=2, fill_alpha=0, line_color='orange')
p.square(md[:,0], md[:,1], legend='MD F&S', size=12, 
         line_width=2, fill_alpha=0, line_color='yellow')
p.legend.location = 'top_left'
show(p)

Repeat using $T^*=0.71$ ($T=85$ K), to match Liquid Argon data published by [Yarnell et al.](Yarnell_Structure_1973.pdf)

In [13]:
T = 0.71

In [14]:
a1 = x[1]*T + x[2]*np.sqrt(T) + x[3] + x[4]/T + x[5]/T**2
a2 = x[6]*T + x[7] + x[8]/T + x[9]/T**2
a3 = x[10]*T + x[11] + x[12]/T
a4 = x[13]
a5 = x[14]/T + x[15]/T**2
a6 = x[16]/T
a7 = x[17]/T + x[18]/T**2
a8 = x[19]/T**2

In [15]:
b1 = x[20]/T**2 + x[21]/T**3
b2 = x[22]/T**2 + x[23]/T**4
b3 = x[24]/T**2 + x[25]/T**3
b4 = x[26]/T**2 + x[27]/T**4
b5 = x[28]/T**2 + x[29]/T**3
b6 = x[30]/T**2 + x[31]/T**3 + x[32]/T**4

In [16]:
pressure071 = (
    density * T + 
    a1 * density**2 + 
    a2 * density**3 +
    a3 * density**4 + 
    a4 * density**5 +
    a5 * density**6 +
    a6 * density**7 +
    a7 * density**8 +
    a8 * density**9 +
    f * b1 * density**3 +
    f * b2 * density**5 +
    f * b3 * density**7 +
    f * b4 * density**9 +
    f * b5 * density**11 +
    f * b6 * density**13    
)

In [17]:
p = figure(width=500, height=500, x_axis_label='Density', 
           y_axis_label='Pressure')
p.line(density, pressure200, legend="T=2.00")
p.line(density, pressure071, legend="T=0.71", line_color="firebrick")
p.legend.location='top_left'
show(p)

While the $T=2.0$ case matches Figure 6.2 in Frenkel & Smit, I am skeptical about the $T=0.71$ case.  I do not understand a negative pressure.  Their article says, "Parameters for the MBWR equation of state for the LJ fluid have been regressed for the temperature range $0.7\leq T^*\leq 6$ and covering the entire fluid range of densities."

### Load in Experimental Data

In [34]:
def rm_par(astr):
    return float(astr.split(b'(')[0])

In [36]:
import sys; print(sys.executable)

/home/schowell/data/myPrograms/anaconda3/envs/sassie/bin/python


In [37]:
data = np.genfromtxt('state_point.dat', dtype=None, names=True, converters={2:rm_par, 3:rm_par})

In [62]:
t20 = data[data['T']==2.0]
t14 = data[data['T']==1.4]
t10 = data[data['T']==1.0]
t07 = data[data['T']==0.7]

In [64]:
p = figure(width=500, height=500, x_axis_label='Density', 
           y_axis_label='Pressure')
p.line(density, pressure200, legend="T=2.00 EOS")
p.line(density, pressure071, legend="T=0.71 EOS", line_color="firebrick")
p.circle(t20['p'], t20['P'], legend="T=2.00 Data")
p.circle(t14['p'], t14['P'], legend="T=1.40 Data", color="green")
p.circle(t10['p'], t10['P'], legend="T=1.00 Data", color="orange")
p.circle(t07['p'], t07['P'], legend="T=0.70 Data", color="firebrick")
p.legend.location='top_left'
show(p)

So the curve matches the experimental data, and the experimental curve even goes negative.

In [59]:
data['T']

array([ 6.  ,  6.  ,  6.  ,  6.  ,  6.  ,  6.  ,  6.  ,  6.  ,  6.  ,
        6.  ,  6.  ,  6.  ,  6.  ,  5.  ,  5.  ,  5.  ,  5.  ,  5.  ,
        5.  ,  5.  ,  5.  ,  5.  ,  5.  ,  5.  ,  5.  ,  4.  ,  4.  ,
        4.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,  4.  ,
        4.  ,  3.  ,  3.  ,  3.  ,  3.  ,  3.  ,  3.  ,  3.  ,  3.  ,
        3.  ,  3.  ,  3.  ,  2.5 ,  2.5 ,  2.5 ,  2.5 ,  2.5 ,  2.5 ,
        2.5 ,  2.5 ,  2.5 ,  2.5 ,  2.5 ,  2.  ,  2.  ,  2.  ,  2.  ,
        2.  ,  2.  ,  2.  ,  2.  ,  2.  ,  2.  ,  2.  ,  1.8 ,  1.8 ,
        1.8 ,  1.8 ,  1.8 ,  1.8 ,  1.8 ,  1.8 ,  1.8 ,  1.8 ,  1.8 ,
        1.6 ,  1.6 ,  1.6 ,  1.6 ,  1.6 ,  1.6 ,  1.6 ,  1.6 ,  1.6 ,
        1.6 ,  1.4 ,  1.4 ,  1.4 ,  1.4 ,  1.4 ,  1.4 ,  1.4 ,  1.4 ,
        1.4 ,  1.4 ,  1.3 ,  1.3 ,  1.3 ,  1.3 ,  1.3 ,  1.3 ,  1.3 ,
        1.3 ,  1.3 ,  1.2 ,  1.2 ,  1.2 ,  1.2 ,  1.2 ,  1.2 ,  1.2 ,
        1.2 ,  1.15,  1.15,  1.15,  1.15,  1.15,  1.15,  1.15,  1.15,
        1.1 ,  1.1 ,

### Compare Experimental Results

In [122]:
N = 2048
T = 0.71
P_runs = ['0p10', '0p11', '0p12', '0p13', '0p14', 
          '0p15', '0p16', '0p17', '0p18', '0p21', '0p23', 
          '0p25', '0p27']
P = [float(val.replace('p', '.')) for val in P_runs]

In [154]:
run_path = ['..', '..', 'simulations', 'lj_sphere_monomer', 'runs']

D = []
p = figure(width=500, height=500, x_axis_label='Steps', 
           y_axis_label='Density')
for P_val, color in itertools.izip(P_runs, itertools.cycle(Viridis6)):
    path = op.join(*[run_path + ['p_{}'.format(P_val), 'output', 'box_length.txt']][0])
    P_dat = np.genfromtxt(path, names=True)
    this_D = P_dat['density'][1000:].mean()
    D.append(this_D)
    p.line(P_dat['step'], P_dat['density'], color=color, alpha=0.05)
    p.line(P_dat['step'][1000:], this_D, legend=P_val, color=color)
    
    p.legend.location='bottom_right'
show(p)

In [169]:
t07

array([(0.005, 0.7, 0.00333, -0.0596, 0.00333, -0.0606, 40000, 0.004),
       (0.005, 0.7, 0.00332, -0.062, 0.00334, -0.058, 150000, 0.006),
       (0.8, 0.7, -0.525, -5.815, -0.357, -5.606, 20000, 0.004),
       (0.9, 0.7, 1.14, -6.429, 1.35, -6.193, 20000, 0.004)], 
      dtype=[('p', '<f8'), ('T', '<f8'), ('P', '<f8'), ('U', '<f8'), ('P_cs', '<f8'), ('U_cs', '<f8'), ('Steps', '<i8'), ('dt', '<f8')])

In [168]:
p = figure(width=500, height=500, x_axis_label='Density', 
           y_axis_label='Pressure')
p.line(density, pressure200, legend="T=2.00 EOS")
p.line(density, pressure071, legend="T=0.71 EOS", line_color="firebrick")
p.circle(t20['p'], t20['P'], legend="T=2.00 Data")
p.circle(t14['p'], t14['P'], legend="T=1.40 Data", color="green")
p.circle(t10['p'], t10['P'], legend="T=1.00 Data", color="orange")
p.circle(t07['p'], t07['P'], legend="T=0.70 Data", color="firebrick")
p.circle(D, P, legend="simulations", color='violet')
p.legend.location='top_left'
show(p)

In [143]:
p = figure()
x = np.linspace(0, 10)
colors = itertools.cycle(Viridis6)
for m, color in itertools.izip(xrange(10), colors):
    y = m * x
    p.line(x, y, legend='m = {}'.format(m), color=color, line_width=2)
    
show(p)

In [159]:
import seaborn.apionly as sns

/home/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [164]:
from bokeh.charts import Area

def create_area_chart(data, palette):
    return Area(data,
                title=palette,
                stack=True,
                palette=palette,
                legend=None,
                xlabel='',
                ylabel='',
                xgrid=False,
                ygrid=False,
                tools='')

In [167]:
palette = sns.palettes.color_palette('colorblind', 9)
palette = palette.as_hex()
data = np.random.random_integers(low=5, high=13, size=[9, 20])
area_charts = [create_area_chart(data, palette), create_area_chart(data, Viridis6)]
grid = gridplot(area_charts, ncols=3, plot_width=300, plot_height=300)
show(grid)

/home/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This function is deprecated. Please call randint(5, 13 + 1) instead
  app.launch_new_instance()


ValueError: expected a value of type basestring, got [u'#0072b2', u'#009e73', u'#d55e00', u'#cc79a7', u'#f0e442', u'#56b4e9', u'#0072b2', u'#009e73', u'#d55e00'] of type _ColorPalette

In [80]:
op.join(*[run_path + [str(p_val)]][0])

'../../simulations/lj_sphere_monomer/runs/0.01'

In [95]:
'p_{:02}'.format(0.20)

'p_0.2'